# Halloween CSV to API db Initial Load POC

## Initial Set-up

In [ ]:
import pandas as pd
import datetime
import os
from etls.dbHelpers import createEngine
import uuid
import numpy as np
import warnings
from pangres import upsert
import requests
import zipfile

warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
warnings.simplefilter(
    action='ignore', category=pd.errors.SettingWithCopyWarning)


### Define helper functions

In [ ]:

def convertBool(val):
    """
    The Halloween CSVs use 1 and 0 to represent True and False. This function maps the 1s to True and the 0s to False.
    """
    primary_to_bool = {1: True, 0: False}
    if val in primary_to_bool.keys():
        return primary_to_bool[val]
    else:
        return False

def show_or_load(df, table_name, schema_name, engine, load=False):
    """
    This function allows you to decide whether you do or don't proceed with loading data (so you can focus on preparing it/ debugging).
    It also prints out the name of the table that is being loaded, so you can tell what has been loaded and what is in progress.
    """
    if load:
        print(f'Loading {table_name}')
        df.to_sql(table_name, schema=schema_name, con=engine,
                  if_exists='append', index=False)
    else:
        print(df.head())



### Create a database engine from the settings specified in your .env file

In [43]:
engine = createEngine()

### Define the scratch workspace where the Halloween CSVs are located and where the NPPES Main File will be downloaded

In [44]:
scratch_dir = os.path.join('..','scratch')

## Get data to fill gaps in Halloween CSVs

### We need to load the FIPS state reference data from the target db, to serve as a lookup table between state abbreviations and state codes, because the Halloween CSVs only contain state abbreviations but the db utilizes state codes

In [ ]:
fips_state_df = pd.read_sql('select * from npd.fips_state', con = engine)

### Since the Halloween CSV files do not contain sufficient NPI data, we need to download the latest NPPES main file and get the additional NPI fields that the target db is expecting

In [ ]:
current_date = datetime.datetime.now()
current_month = current_date.strftime("%B")
current_year = current_date.year
csv_version = f'10_2025_V2'
nppes_dir = os.path.join(scratch_dir,'nppes')

# Download and unzip the NPPES CSV files
zipData = requests.get(f'https://download.cms.gov/nppes/NPPES_Data_Dissemination_{csv_version}.zip').content
with zipfile.ZipFile(io.BytesIO(zipData), 'r') as zip_file:
   zip_file.extractall(nppes_dir)
main_files = [f for f in os.listdir(nppes_dir) if 'npidata_pfile' in f and '_fileheader' not in f]
main_files.sort()
latest_main_file = main_files[-1]

npi_df = pd.read_csv(os.path.join(nppes_dir, latest_main_file), usecols = ['Provider Last Name (Legal Name)', 'NPI', 'Entity Type Code', 'Replacement NPI', 'Provider Enumeration Date', 'Last Update Date',
                   'NPI Deactivation Reason Code', 'NPI Deactivation Date', 'NPI Reactivation Date', 'Certification Date'])
npi_df_renamed = npi_df.rename(columns={
        'NPI': 'npi',
        'Entity Type Code': 'entity_type_code',
        'Replacement NPI': 'replacement_npi',
        'Provider Enumeration Date': 'enumeration_date',
        'Last Update Date': 'last_update_date',
        'NPI Deactivation Reason Code': 'deactivation_reason_code',
        'NPI Deactivation Date': 'deactivation_date',
        'NPI Reactivation Date': 'reactivation_date',
        'Certification Date': 'certification_date'
    })

### The NPPES main file strips certain field values for records with deactivated NPIs, so we populate those as needed for the target db.
1. Deactivated NPIs show up without entity_type_code values, but those are required in the db. We use the Provider Last Name (Legal Name) field to intuit whether a provider is an invidual (if there is a last name listed, the provider has entity_type_code 1) or an organization (if there is not a last name listed, the provider has entity_type_code 2)
2. Deactivated NPIs show up without enumeration_date and last_update_date values. We populate bogus dates of 1/1/1900.

In [ ]:

deactivated_npi1_condition = (npi_df_renamed['entity_type_code'].isnull())&~(npi_df_renamed['Provider Last Name (Legal Name)'].isnull())
deactivated_npi2_condition = (npi_df_renamed['entity_type_code'].isnull())&(npi_df_renamed['Provider Last Name (Legal Name)'].isnull())
npi_df_renamed.loc[deactivated_npi1_condition, ['entity_type_code', 'enumeration_date', 'last_update_date']] = [1, '1900-01-01', '1900-01-01']
npi_df_renamed.loc[deactivated_npi2_condition, ['entity_type_code', 'enumeration_date', 'last_update_date']] = [2, '1900-01-01', '1900-01-01']
del npi_df_renamed['Provider Last Name (Legal Name)']

In [50]:
df_dict={}
for f in os.listdir(os.path.join(scratch_dir, 'halloween_data')):
    if '.csv' in f:
        tablename = f.split('.csv')[0]
        df = pd.read_csv(os.path.join(scratch_dir,'halloween_data',f), na_values=[''], keep_default_na=False)
        df_dict[f]=df
        #df.to_sql(tablename, index=False, schema = 'raw_csv', con = engine, if_exists='replace')

In [ ]:
practitioner_df = df_dict['practitioner.csv']
#note: we can do this because each practitioner only appears once in this table
practitioner_df['id'] = [uuid.uuid4() for i in practitioner_df.index]
practitioner_df_renamed = practitioner_df.rename(columns = {'gender_code': 'sex', 'name_prefix': 'prefix', 'name_suffix': 'suffix'})
practitioner_df_renamed['name_use_id'] = 1
practitioner_taxonomy_df = df_dict['practitionerrole.csv']
filtered_practitioner_taxonomy_df = practitioner_taxonomy_df.loc[practitioner_taxonomy_df['practitioner_id']!=1770923773]
merged_taxonomy_df = practitioner_taxonomy_df.merge(practitioner_df_renamed, left_on = 'practitioner_id', right_on = 'npi', suffixes = ('tax', 'individual')) 
merged_taxonomy_df = merged_taxonomy_df.loc[merged_taxonomy_df['state_code']!='ZZ']
merged_taxonomy_df['state_code'] = [fips_state_df.loc[i]['id'] if i in fips_state_df.index else np.nan for i in merged_taxonomy_df['state_code']]
merged_taxonomy_df_renamed = merged_taxonomy_df.rename(columns={'idindividual': 'individual_id', 'taxonomy_code':'nucc_code'})
provider_to_taxonomy_df = merged_taxonomy_df_renamed[['npi', 'nucc_code', 'is_primary']]
provider_to_taxonomy_df['is_primary'] = provider_to_taxonomy_df['is_primary'].apply(lambda x: convertBool(x))
dedup_taxonomy_df = provider_to_taxonomy_df.sort_values(by='is_primary', ascending=False)[
        ['npi', 'nucc_code', 'is_primary']].drop_duplicates(subset=['nucc_code', 'npi'])
dedup_taxonomy_df['id'] = [uuid.uuid4() for i in dedup_taxonomy_df.index]
credential_df = provider_to_taxonomy_df.merge(merged_taxonomy_df_renamed, on = ['npi', 'nucc_code'], suffixes = ('tax', 'cred'))
credential_df_renamed = credential_df.rename(columns={'idtax': 'provider_to_taxonomy_id'})

In [ ]:
organization_df = df_dict['organization.csv']
organization_df['is_primary'] = True
organization_df_renamed = organization_df.rename(columns={'id':'old_org_id', 'parent_id':'old_parent_id', 'organization_name':'name'})
organization_df_renamed.set_index(['old_org_id'], inplace=True)
organization_df_renamed['org_id'] = [uuid.uuid4() for i in organization_df_renamed.index]
organization_df_renamed['org_parent_id'] = [organization_df_renamed.loc[i]['org_id'] if i in fips_state_df.index else np.nan for i in organization_df_renamed['old_parent_id']]
organization_npi_df = df_dict['organization_npi.csv']
organization_npi_df_renamed = organization_npi_df.rename(columns={'organization_id':'old_org_id'})
organization_npi_df_renamed['id'] = [uuid.uuid4() for i in organization_npi_df_renamed.index]
clinical_organization_df = organization_npi_df_renamed.merge(organization_df_renamed, on='old_org_id', how='outer')
clinical_organization_df_renamed = clinical_organization_df.rename(columns={'org_id':'parent_id'})
other_organization_df = organization_df_renamed.rename(columns = {'org_id':'id', 'org_parent_id': 'parent_id'})


In [87]:
endpoint_df = df_dict['endpoint.csv']
endpoint_df_renamed = endpoint_df.rename(columns={'id':'endpoint_id','fhir_url':'address'})
ehr_vendor_df = endpoint_df.drop_duplicates(subset='vendor_name')
ehr_vendor_df['id'] = [uuid.uuid4() for i in ehr_vendor_df.index]
ehr_vendor_df_renamed = ehr_vendor_df.rename(columns={'vendor_name':'name'})
ehr_vendor_df_renamed.set_index('name', inplace=True, drop=False)
endpoint_df_renamed['ehr_vendor_id'] = endpoint_df_renamed['vendor_name'].apply(lambda x: ehr_vendor_df_renamed.loc[x]['id'])
endpoint_df_renamed['environment_type_id'] = 'prod'
endpoint_df_renamed['endpoint_connection_type_id'] = 'hl7-fhir-rest'
endpoint_df_renamed['id'] = [uuid.uuid4() for i in endpoint_df_renamed.index]

In [88]:
org_to_endpoint_df = df_dict['organization_endpoint.csv']
merged_org_to_endpoint_df = org_to_endpoint_df.merge(endpoint_df_renamed, on = 'endpoint_id', how='outer').merge(clinical_organization_df_renamed, left_on = 'organization_npi', right_on = 'npi', suffixes = ('endpoint', 'organization'), how='outer')
merged_org_to_endpoint_df= merged_org_to_endpoint_df[['idendpoint', 'idorganization']].rename(columns = {'idendpoint': 'endpoint_instance_id', 'idorganization':'organization_id'})

In [ ]:
address_df = df_dict['location.csv']
address_df_renamed = address_df.rename(columns={'id':'address_us_id', 'line':'delivery_line_1', 'postalcode':'zipcode', 'city':'city_name'})
address_df_renamed['id']= [uuid.uuid4() for i in address_df_renamed.index]
address_df_renamed = address_df_renamed.loc[(address_df_renamed['state'] != 'FM') & (address_df_renamed['state'] != '~') & (address_df_renamed['state'] != 'UK') & (address_df['state'] != 'MH')]
address_df_renamed['state_code'] = address_df_renamed['state'].apply(lambda x: fips_state_df.loc[x]['id'])
location_npi_df = df_dict['npi_location.csv']
merged_df_1 = location_npi_df.merge(address_df_renamed, left_on='location_id', right_on = 'address_us_id', how='outer')
merged_df_2 = merged_df_1.merge(npi_df_renamed, on = 'npi', suffixes=('address','npi'), how='outer')
merged_df_3 = merged_df_2.merge(practitioner_df_renamed, on = 'npi', suffixes = ('address', 'individual'), how='outer')
merged_location_df = merged_df_3.merge(clinical_organization_df_renamed, on = 'npi', suffixes = ('address', 'organization'), how='outer')
merged_location_df_renamed = merged_location_df.rename(columns={'idaddress':'address_id', 'idindividual':'individual_id', 'id':'organization_id', 'nameaddress':'name'})
merged_location_df_renamed['address_use_id'] = 2
individual_to_address_df = merged_location_df_renamed[['address_id','individual_id', 'address_use_id']].dropna(how='any')
location_df = merged_location_df_renamed[['address_id','organization_id','name', 'address_use_id']].dropna(how='any')
location_df['id'] = [uuid.uuid4() for i in location_df.index]
location_to_endpoint_df = location_df.merge(merged_org_to_endpoint_df, on = 'organization_id', how='outer')[['id', 'endpoint_instance_id']].dropna(how = 'any').rename(columns = {'id':'location_id'})


In [153]:
provider_to_organization_df = df_dict['personal_npi_to_organizational_npi.csv']
merged_provider_to_org_df = provider_to_organization_df.merge(practitioner_df_renamed, left_on = 'personal_npi', right_on = 'npi', how='inner').merge(clinical_organization_df_renamed, left_on = 'organizational_npi', right_on = 'npi', suffixes = ('individual', 'organization'), how='inner')
provider_to_org_df_renamed = merged_provider_to_org_df.rename(columns = {'idindividual':'individual_id', 'idorganization':'organization_id'})
provider_to_org_df_renamed['id'] = [uuid.uuid4() for i in provider_to_org_df_renamed.index]
provider_to_org_df_renamed['relationship_type_id'] = [2 if val=='PECOS Assignment Relationships' else val for val in provider_to_org_df_renamed['affiliation_source']]
provider_to_location_df = provider_to_org_df_renamed.merge(location_df, on='organization_id', how='inner', suffixes=('porg','location'))
provider_to_location_df['id'] = [uuid.uuid4() for i in provider_to_location_df.index]
provider_to_location_df_renamed = provider_to_location_df.rename(columns={'idlocation':'location_id', 'idporg':'provider_to_organization_id'})

In [163]:
schema_name = 'npd'
load = True

# load npi
#show_or_load(npi_df_renamed, 'npi', schema_name, load)

# load individual
#show_or_load(practitioner_df_renamed[['id', 'sex']], 'individual', schema_name, load)
practitioner_df_renamed_renamed = practitioner_df_renamed.rename(columns={'id':'individual_id'})

# load individual_to_name
#show_or_load(practitioner_df_renamed_renamed[['individual_id', 'first_name', 'middle_name', 'last_name', 'prefix', 'suffix', 'name_use_id']], 'individual_to_name', schema_name, load)

# load provider
#show_or_load(practitioner_df_renamed_renamed.merge(npi_df_renamed, on = 'npi', how='inner')[['npi', 'individual_id']], 'provider', schema_name, load)

# load organization
#show_or_load(other_organization_df[['id']], 'organization', schema_name, load)
other_organization_df.set_index('id', drop=False, inplace=True)
#######upsert(df = other_organization_df[['parent_id']], con = engine, schema = schema_name, if_row_exists='update', table_name = 'organization')
#show_or_load(clinical_organization_df_renamed[['id']], 'organization', schema_name, load)
clinical_organization_df_renamed.set_index('id', drop=False, inplace=True)
#######upsert(df = clinical_organization_df_renamed[['parent_id']], con = engine, schema = schema_name, if_row_exists='update', table_name = 'organization')

other_organization_df_renamed = other_organization_df.rename(columns={'id':'organization_id', 'organization_name':'name'})
clinical_organization_df_renamed_renamed = clinical_organization_df_renamed.rename(columns={'id':'organization_id'})

# load organization_to_name
#show_or_load(other_organization_df_renamed[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)
#show_or_load(clinical_organization_df_renamed_renamed[['organization_id', 'name', 'is_primary']], 'organization_to_name', schema_name, load)

# load clinical_organization
#show_or_load(clinical_organization_df_renamed_renamed[['organization_id', 'npi']], 'clinical_organization', schema_name, load)

# load ehr_vendor
#show_or_load(ehr_vendor_df_renamed[['id', 'name']], 'ehr_vendor', schema_name, load)

# load endpoint_instance
#show_or_load(endpoint_df_renamed[['id', 'ehr_vendor_id', 'address', 'endpoint_connection_type_id', 'environment_type_id']], 'endpoint_instance', schema_name, load)

# load address_us
#show_or_load(address_df_renamed[['address_us_id', 'delivery_line_1','city','state_code','zipcode']].rename(columns={'address_us_id':'id', 'city':'city_name'}), 'address_us', schema_name, load)

# load address
#show_or_load(address_df_renamed[['id', 'address_us_id']], 'address', schema_name, load)

# load individual_to_address
#show_or_load(individual_to_address_df, 'individual_to_address', schema_name, load)

# load organization_to_address
#show_or_load(location_df[['address_id','organization_id', 'address_use_id']], 'organization_to_address', schema_name, load)

# load location

#show_or_load(location_df[['id','address_id','organization_id']], 'location', schema_name, load)

# load location_to_endpoint
#show_or_load(location_to_endpoint_df, 'location_to_endpoint', schema_name, load)

# load provider_to_organization
#show_or_load(provider_to_org_df_renamed[['individual_id', 'organization_id', 'relationship_type_id','id']], 'provider_to_organization', schema_name, load)

# load provider_to_location
#show_or_load(provider_to_location_df_renamed[['location_id', 'provider_to_organization_id', 'id']], 'provider_to_location', schema_name, load)

# load provider_to_taxonomy
show_or_load(filtered_dedup_taxonomy_df, 'provider_to_taxonomy', schema_name, load)

# load provider_to_credential
show_or_load(credential_df_renamed[['license_number', 'state_code', 'provider_to_taxonomy_id']], 'provider_to_credential', schema_name, load)

Loading provider_to_taxonomy
Loading provider_to_credential


ProgrammingError: (psycopg2.errors.DatatypeMismatch) column "provider_to_taxonomy_id" is of type uuid but expression is of type integer
LINE 1: ...e, provider_to_taxonomy_id) VALUES ('2744', '20', 3), ('RT00...
                                                             ^
HINT:  You will need to rewrite or cast the expression.

[SQL: INSERT INTO npd.provider_to_credential (license_number, state_code, provider_to_taxonomy_id) VALUES (%(license_number__0)s, %(state_code__0)s, %(provider_to_taxonomy_id__0)s), (%(license_number__1)s, %(state_code__1)s, %(provider_to_taxonomy_id__1)s) ... 81418 characters truncated ... axonomy_id__998)s), (%(license_number__999)s, %(state_code__999)s, %(provider_to_taxonomy_id__999)s)]
[parameters: {'provider_to_taxonomy_id__0': 3, 'license_number__0': '2744', 'state_code__0': '20', 'provider_to_taxonomy_id__1': 11, 'license_number__1': 'RT006860', 'state_code__1': '42', 'provider_to_taxonomy_id__2': 50, 'license_number__2': 'PA190185', 'state_code__2': '4

In [ ]:
practitioner_taxonomy_df = df_dict['practitionerrole.csv']
filtered_practitioner_taxonomy_df = practitioner_taxonomy_df.loc[practitioner_taxonomy_df['practitioner_id']!=1770923773]
merged_taxonomy_df = practitioner_taxonomy_df.merge(practitioner_df_renamed, left_on = 'practitioner_id', right_on = 'npi', suffixes = ('tax', 'individual')) 
merged_taxonomy_df = merged_taxonomy_df.loc[merged_taxonomy_df['state_code']!='ZZ']
merged_taxonomy_df['state_code'] = [fips_state_df.loc[i]['id'] if i in fips_state_df.index else np.nan for i in merged_taxonomy_df['state_code']]
merged_taxonomy_df_renamed = merged_taxonomy_df.rename(columns={'idindividual': 'individual_id', 'taxonomy_code':'nucc_code'})
provider_to_taxonomy_df = merged_taxonomy_df_renamed[['npi', 'nucc_code', 'is_primary']]
provider_to_taxonomy_df['is_primary'] = provider_to_taxonomy_df['is_primary'].apply(lambda x: convertBool(x))
dedup_taxonomy_df = provider_to_taxonomy_df.sort_values(by='is_primary', ascending=False)[
        ['npi', 'nucc_code', 'is_primary']].drop_duplicates(subset=['nucc_code', 'npi'])
dedup_taxonomy_df['id'] = [uuid.uuid4() for i in dedup_taxonomy_df.index]
credential_df = provider_to_taxonomy_df.merge(merged_taxonomy_df_renamed, on = ['npi', 'nucc_code'], suffixes = ('tax', 'cred'))
credential_df_renamed = credential_df.rename(columns={'idtax': 'provider_to_taxonomy_id'})

In [166]:
practitioner_taxonomy_df

,id,practitioner_id,taxonomy_code,state_code,license_number,is_primary
0,1,1497203558,2081P2900X,MD,D0068884,1
1,2,1306394366,1041C0700X,CO,CSW.09923802,1
2,3,1487102463,103T00000X,KS,2744,0
3,4,1396293387,106H00000X,CA,MFC#78448,1
4,5,1740738707,363LP0808X,NC,180906,1
...,...,...,...,...,...,...
7293953,7293954,1174071229,122300000X,NC,8025,1
7293954,7293955,1326596479,207YX0901X,VA,0101254181,1
7293955,7293956,1851849863,207Q00000X,CA,A102339,1
7293956,7293957,1639627664,207P00000X,HI,18693,1
